# Код для веб-скрейпинга данных о пассажирах Титаника с Wikipedia

In [12]:
import numpy as np
import pandas as pd

import requests
import urllib.request
from urllib.request import urlopen

from bs4 import BeautifulSoup

In [13]:
url = 'https://en.wikipedia.org/wiki/Passengers_of_the_Titanic'
html = urlopen(url) 
soup = BeautifulSoup(html, 'html.parser')

Изучив содержимое HTML страницы, понимаем, что нужные нам данные сидят в тегах **table class="wikitable sortable"**.
Указав тег, находим все таблицы на странице и выбираем только нужные нам (First class, Second class, Third class)

In [14]:
all_tables = soup.find_all('table', class_='wikitable sortable')
table_1 = all_tables[0]
table_2 = all_tables[1]
table_3 = all_tables[2]

Еще раз изучив HTML страницу, понимаем, что строки в талицах разделяются с помощью элемента **'tr'**, а значения в ячейках с помощью **'td'**.


Информацию передадим в списки, но т.к. функция find_all() не работает одновременно с неколькими элементами, то делаем все для каждой таблицы отдельно (из-за этого три отдельных цикла).

In [15]:
name=[]
age=[]

for row in table_1.find_all('tr'):
    cells = row.find_all('td')
    if len(cells)>1: # Прописываем условие чтобы не вытаскивать заголовок
        name.append(cells[0].find(text=True))
        age.append(cells[1].find(text=True))

for row in table_2.find_all('tr'):
    cells = row.find_all('td')
    if len(cells)>1: # Прописываем условие чтобы не вытаскивать заголовок
        name.append(cells[0].find(text=True))
        age.append(cells[1].find(text=True))
        
for row in table_3.find_all('tr'):
    cells = row.find_all('td')
    if len(cells)>1: # Прописываем условие чтобы не вытаскивать заголовок
        name.append(cells[0].find(text=True))
        age.append(cells[1].find(text=True))

Помещаем всю собранную информацию в датафрейм и смотрим что получилось.

In [16]:
df = pd.DataFrame(name, columns=['name'])
df['age'] = age
df.head(17)

,name,age
0,"Allen, Miss Elizabeth Walton",29
1,"Allison, Mr. Hudson Joshua Creighton",30
2,"and chauffeur, Mr. George Swane",19
3,"and cook, Miss Amelia Mary ""Mildred"" Brown",18
4,"Allison, Mrs. Bessie Waldo (née Daniels)",25
5,"and maid, Miss Sarah Daniels",33
6,"Allison, Miss Helen Loraine",2
7,"Allison, Master Hudson Trevor",11 mo.
8,"and nurse, Miss Alice Catherine Cleaver",22
9,"Anderson, Mr. Harry",47


**Сравниваем на полноту и точность со страницей на Википедии.**


Похоже, что имена собрали нормально и возраст совпадает, но есть нюансы: 
* Для детей до года, возраст указывался в месяцах с пометкой 'mo.'
* Есть пассажиры для которых вместе с возрастом подтянулся перенос строки '\n'
* Для нескольких людей вместо возраста подтянулся город из-за форматирования таблицы на сайте. Так как их немного, думаю, что мы просто заменим значения на 30 (возраст взрослого человека), чтобы сильно не влиять на выживаемость.

In [17]:
# Заменим для детей до года возраст на 1 (год), чтобы все было единообразно.
df.age = df.age.apply(lambda x: '1' if 'mo.' in x else x)

# Убираем перенос строки 
df.age = df.age.apply(lambda x: x.replace('\n', ''))

# Убираем города из возраста 
df.age = df.age.apply(lambda x: '30' if len(x) > 2 else x)

In [20]:
df.head(17)

,name,age
0,"Allen, Miss Elizabeth Walton",29
1,"Allison, Mr. Hudson Joshua Creighton",30
2,"and chauffeur, Mr. George Swane",19
3,"and cook, Miss Amelia Mary ""Mildred"" Brown",18
4,"Allison, Mrs. Bessie Waldo (née Daniels)",25
5,"and maid, Miss Sarah Daniels",33
6,"Allison, Miss Helen Loraine",2
7,"Allison, Master Hudson Trevor",1
8,"and nurse, Miss Alice Catherine Cleaver",22
9,"Anderson, Mr. Harry",47


Проверяем на всякий случай еще раз что данные без пропусков

In [19]:
df.isna().sum()

name    0
age     0
dtype: int64

Сохраняем данные в csv-файл

In [75]:
df.to_csv(r'C:\Users\vakhr\Desktop\scraped_age.csv')